BLG 453E - Computer Vision<br>Homework 4<br>Muhammed Tolga Cangöz<br>150130024

In [31]:
from PIL import Image
from torchvision import transforms
import torch
import torchvision.models as models
import os, glob
import numpy as np
import csv

def obtain_features(model, img):
	preprocess = transforms.Compose([
		transforms.Resize(256),
		transforms.CenterCrop(224),
		transforms.ToTensor(),
		transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
	])
	input_tensor = preprocess(img)
	input_batch = input_tensor.unsqueeze(0)

	with torch.no_grad():
		output = model(input_batch.cuda())


	return output


def prepare_train_from_folder(dir, model):

	img_count = len(glob.glob(dir +'/*/*.JPEG'))

	train_folders = sorted(os.listdir(dir))

	train = np.zeros((img_count, 4096))
	train_labels = np.zeros((img_count,))

	counter = 0

	for i in range(len(train_folders)):
		all_imgs = os.listdir(dir + '/' + train_folders[i])

		for j in range(len(all_imgs)):
			img = Image.open(dir + '/' + train_folders[i] + '/' + all_imgs[j])
			img = img.convert('RGB')

			features = obtain_features(model, img)

			train[counter, :] = features.cpu().numpy()
			train_labels[counter] = i

			counter += 1

	return train, train_labels


def prepare_test_from_folder(dir, model):

	img_count = len(glob.glob(dir + '/*.JPEG'))

	test_files = sorted(os.listdir(dir))

	test = np.zeros((img_count, 4096))
	test_names = []

	counter = 0

	for i in range(len(test_files)):
		img = Image.open(dir +  '/' + test_files[i])
		img = img.convert('RGB')

		features = obtain_features(model, img)

		test[counter, :] = features.cpu().numpy()
		counter += 1

		test_names.append(test_files[i])

	return test, test_names




train_dir = 'imagenet_50/train'
test_dir = 'imagenet_50/test/imgs'

In [ ]:
model = torch.load('model_2.pth')
model.cuda().eval()

In [3]:
test1, test_names = prepare_test_from_folder(test_dir, model)

train1, train_labels = prepare_train_from_folder(train_dir, model)

In [32]:
test2, test_names = prepare_test_from_folder(test_dir, model)

train2, train_labels = prepare_train_from_folder(train_dir, model)

In [4]:
# Model_1.pth-only submission due to the limited computing power, but other models will be used later as well for the final submission.
test1.shape

(50000, 512)

In [10]:
import os
names = sorted(os.listdir("imagenet_50/train"))

In [26]:
train1.shape

(35000, 512)

In [22]:
from sklearnex import patch_sklearn
patch_sklearn()

from sklearn.neighbors import KNeighborsClassifier

from sklearn.decomposition import PCA
pca = PCA(n_components=300)
pca.fit(train1)
train1_pca = pca.transform(train1)
test1_pca = pca.transform(test1)

In [27]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train1)
train1_scaled = scaler.transform(train1)
test1_scaled = scaler.transform(test1)

In [28]:
knn = KNeighborsClassifier(n_neighbors=120, n_jobs=-1)
knn.fit(train1_scaled, train_labels)
predicted_labels = knn.predict(test1_scaled)


predicted_last = [0]*len(test_names)
for i in range(len(test_names)):
    predicted_last[i] = names[int(predicted_labels[i])]

with open('submission_baseline_pca_300_norm.csv', mode='w') as submission:
    submission_writer = csv.writer(submission, delimiter=',')
    submission_writer.writerow(["FileName","Class"])
    
    for i in range(len(test_names)):
        submission_writer.writerow([test_names[i], predicted_last[i]])